In [13]:
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [14]:
import pandas as pd
df = pd.read_csv("/content/Drive/MyDrive/output1.txt", sep="|", header=None, names=["text", "rephrase"])
# print(df.head())

#df = pd.read_csv("./output.txt", sep="|", header=None, names=["text", "rephrase"])
print(df.head())

                                                text  \
0  I need to get away from this city for a bit. I...   
1  Hate when people try to guilt trip you into so...   
2  Why does it have to be so hard to network if y...   
3  Children with autism use stereotypic behavior ...   
4  I have less than 2 hours to get home and make ...   

                                            rephrase  
0  I'm thankful for everything this city gave me!...  
1  I don’t like it when people try to guilt -trip...  
2  I'm glad networking with gaming comes very eas...  
3  Children with autism tend to demonstrate stere...  
4  I have less than 2 hours to get home and make ...  


In [4]:
!pip install transformers
!pip install Datasets
!pip install evaluate
!pip3 install rouge_score
!pip3 install sacrebleu
!pip install transformers -U
!pip install accelerate -U
#!pip install torch -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f4bf1684ec88d50c1849b3fc8fc06b74806501551165cb4610c488adc6863e44
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.0 MB/s eta 0:00:00


In [5]:
import argparse
import pandas as pd
import numpy as np
import random
#from sentence_transformers import SentenceTransformer, util
import os
from transformers import Trainer, pipeline, set_seed, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
nltk.download('punkt')
import csv
from datasets import load_dataset, load_metric, DatasetDict, Dataset


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
# Example data preparation
input_texts = ["The quick brown fox.", "A sentence to be rephrased."]
positive_rephrases = ["A fast brown fox.", "A differently phrased sentence."]

input_encodings = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
positive_encodings = tokenizer(positive_rephrases, return_tensors="pt", padding=True, truncation=True)


In [6]:
from sklearn.model_selection import train_test_split

# Assuming your dataframe is named 'df'
# Specify the test size as 0.05 to get a 95%/5% train/test split
train_df, test_df = train_test_split(df, test_size=0.10, random_state=42)

In [7]:
train_data = train_df.iloc[0:4000]
val_data = train_df.iloc[4000:6011]

In [8]:
# convert to dataset
raw_data = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "eval": Dataset.from_pandas(val_data),
    "test": Dataset.from_pandas(test_df)
})

In [11]:
metric = load_metric("rouge")
metric2 = load_metric("sacrebleu")

<ipython-input-11-6637e6a9532a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [12]:
def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Rouge expects a newline after each sentence
        decoded_preds_joined = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels_joined = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

        result = metric.compute(predictions=decoded_preds_joined, references=decoded_labels_joined, use_stemmer=True)
        # Extract a few results
        #result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
        result = {key: value.mid.fmeasure for key, value in result.items()}

        # Add mean generated length
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)

        # Add bleu scores
        decoded_labels_expanded = [[x] for x in decoded_labels]
        result2 = metric2.compute(predictions=decoded_preds, references=decoded_labels_expanded)
        result['sacrebleu'] = round(result2["score"], 1)

        return {k: round(v, 4) for k, v in result.items()}

In [13]:
def preprocess_function(row):
    inputs = row["text"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(row["rephrase"], max_length=64, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenize_data = raw_data.map(preprocess_function, batched = True, remove_columns=['text', 'rephrase'])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2011 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
batch_size = 1
# Example fine-tuning configuration
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/Drive/MyDrive/models-nlp/output/bart-rephrase",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size= batch_size,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    overwrite_output_dir=True,
    fp16=False,
    logging_dir="./logs",
)




In [17]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['eval'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)


In [18]:
trainer.train()
trainer.save_model("/content/Drive/MyDrive/models-nlp/output/bart-rephrase")

Step,Training Loss
500,1.197200
1000,0.947100
1500,0.905900
2000,0.922300
2500,0.858700
3000,0.869300
3500,0.862400
4000,0.844700
4500,0.560400
5000,0.568800


In [20]:
trainer.save_model("/content/Drive/MyDrive/models-nlp/output/bart-rephrase1")

In [19]:
trainer.evaluate()

{'eval_loss': 0.8409807682037354,
 'eval_rouge1': 0.2767,
 'eval_rouge2': 0.1091,
 'eval_rougeL': 0.2113,
 'eval_rougeLsum': 0.2356,
 'eval_gen_len': 60.2029,
 'eval_sacrebleu': 5.6,
 'eval_runtime': 2782.5707,
 'eval_samples_per_second': 0.723,
 'eval_steps_per_second': 0.723,
 'epoch': 2.0}

In [16]:
from transformers import BartForConditionalGeneration, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/Drive/MyDrive/models-nlp/output/bart-rephrase")
tokenizer = AutoTokenizer.from_pretrained("/content/Drive/MyDrive/models-nlp/output/bart-rephrase")
positivereframer = pipeline('summarization', model=model, tokenizer=tokenizer)
texts = raw_data['test']['text']

OSError: ignored

In [23]:
positivereframer("Southend Hospital was like a cross between Argos and Southendairport today Everybody looked SO and sad! NOT a place toGET well")

Your max_length is set to 142, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


[{'summary_text': "The hospital was like a cross between Argos and Southendairport today. Everybody looked SO and sad, but I'm thankful that they were able to care for the sick and sick people in the hospital, I'm sure they'll get better soon."}]

In [ ]:
reframed_phrases = [positivereframer(text)[0]['summary_text'] for text in texts[:20]]

In [17]:
reframed_phrases

NameError: ignored

In [29]:
with open(os.path.join("/content/Drive/MyDrive/models-nlp/output/bart-output.txt"), 'w') as f:
        for item in reframed_phrases:
            f.write("%s\n" % item)

In [11]:
texts = raw_data['test']['text']

with open(os.path.join("/content/Drive/MyDrive/models-nlp/output/bart-input.txt"), 'w') as f:
        for item in texts[:20]:
            f.write("%s\n" % item)

In [9]:
texts = raw_data['test']['rephrase']

with open(os.path.join("/content/Drive/MyDrive/models-nlp/output/bart-original-rephrased.txt"), 'w') as f:
        for item in texts[:20]:
            f.write("%s\n" % item)